<a href="https://colab.research.google.com/github/Andbwol/Lenta.Ru-News-Dataset/blob/master/Homework_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import joblib as pickle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

Были большие проблемы с загрузкой датасета lenta-ru-news.
Архив в итоге брал вот отсюда:
https://github.com/yutkin/Lenta.Ru-News-Dataset/releases


 Почему-то не хотел разархивироваться архив через bz2. Выдавал ошибку. Пришлось распаковать файл локально и загружать во временное хранилище. При этом при переводе csv в df pandas постоянно вылетают ошибки чтения utf-8 и EOF в разных рядах. Удалось скормить вот в таком виде (см ниже). Это также позволило уменьшить объем базы и сократить время обучения.
В целом удивило, что с лемматизацией F1 ухудшился. Возможно как-раз из-за сжатого размера базы.

In [ ]:
df=pd.read_csv('lenta-ru-news.csv', error_bad_lines=False, encoding='utf-8', skiprows = [i for i in range(40000,800000)])

<ipython-input-34-30c21cd67f25>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('lenta-ru-news.csv', error_bad_lines=False, encoding='utf-8', skiprows = [i for i in range(40000,800000)])


In [ ]:
df.shape

(39999, 6)

In [ ]:
df.topic.value_counts()

Россия       16585
Мир           9066
Экономика     4727
Культура      2789
Спорт         2056
Name: topic, dtype: int64

In [ ]:
df.head()

,url,title,text,topic,tags,date
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31


In [ ]:
df = df[df.topic.isin(['Россия','Мир','Экономика','Спорт','Культура'])].copy()

In [ ]:
df.shape

(35223, 6)

In [ ]:
df.isna().sum()

url      0
title    0
text     0
topic    0
tags     0
date     0
dtype: int64

In [45]:
df[df.text.isna()]['title'].values[0]

IndexError: ignored

In [46]:
df.dropna(subset=['text'],inplace = True)

In [47]:
df.topic.value_counts()

Россия       16585
Мир           9066
Экономика     4727
Культура      2789
Спорт         2056
Name: topic, dtype: int64

In [48]:
df.topic.value_counts(normalize=True)

Россия       0.470857
Мир          0.257389
Экономика    0.134202
Культура     0.079181
Спорт        0.058371
Name: topic, dtype: float64

In [49]:
df.isna().sum()

url      0
title    0
text     0
topic    0
tags     0
date     0
dtype: int64

In [50]:
df[df.text.isna()]['title'].values

array([], dtype=object)

In [ ]:
# df = df[~df.text.isna()]

In [51]:
df.shape

(35223, 6)

In [52]:
RANDOM_STATE = 42

### Базовая обработка

In [53]:
text_transformer = TfidfVectorizer()

In [54]:
%%time
text = text_transformer.fit_transform(df['text'])

CPU times: user 7.33 s, sys: 111 ms, total: 7.44 s
Wall time: 7.55 s


In [55]:
df['text']

5        Как стало известно агентству Ассошиэйтед Пресс...
6        В зале игровых автоматов в третьем ярусе подзе...
7        Япония приняла решение разморозить кредиты Рос...
8        Британцы отмечают сегодня скорбную дату - втор...
9        В понедельник директор департамента внешних св...
                               ...                        
39994    Два тура президентских выборов во Франции прой...
39995    В Москве прошла конференция "РАО "ЕЭС России" ...
39996    Во вторник Верховный суд Якутии возобновил про...
39997    Во вторник Верховный суд Якутии отменил регист...
39998    Глава ГТК Михаил Ванин считает возможным увели...
Name: text, Length: 35223, dtype: object

In [56]:
text.shape

(35223, 220790)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [58]:
X_train.shape

(28178, 220790)

In [59]:
X_test.shape

(7045, 220790)

In [60]:
%%time
clf = tree.DecisionTreeClassifier()

CPU times: user 32 µs, sys: 3 µs, total: 35 µs
Wall time: 38.1 µs


In [61]:
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [62]:
pred = clf.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

Качество модели по метрике F1 0.7211157243634333


### Изменим токенизатор

In [64]:
import nltk

In [65]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [66]:
stop_russian = stopwords.words('russian')

In [67]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

In [68]:
from sklearn.ensemble import GradientBoostingClassifier

In [69]:
X_train.shape

(28178, 10000)

In [70]:
clf2 = GradientBoostingClassifier()
clf2.fit(X_train,y_train)

GradientBoostingClassifier()

In [71]:
print('TRAIN - Качество модели по метрике F1 (токенизатор настроен)',
      f1_score(y_train,clf2.predict(X_train),average='weighted'))

TRAIN - Качество модели по метрике F1 (токенизатор настроен) 0.8492956747551531


In [72]:
%%time
pred2 = clf2.predict(X_test)
print('TEST - Качество модели по метрике F1 (токенизатор настроен)', f1_score(y_test,pred2,average='weighted'))

TEST - Качество модели по метрике F1 (токенизатор настроен) 0.8197913283785708
CPU times: user 127 ms, sys: 0 ns, total: 127 ms
Wall time: 132 ms


In [73]:
with open ('transformer.pkl', 'wb') as f:
    pickle.dump(text_transformer, f)

In [74]:
with open ('clf.pkl', 'wb') as f:
    pickle.dump(clf2, f)

### приведем слова к первой форме

In [75]:
pattern = r'[^А-Яа-я]+'

In [76]:
list = [re.sub(pattern, ' ', i) for i in ['проверкаyyy weyweywt iweiwue ><<++ 238238 ? 123 тест']]

In [77]:
list

['проверка тест']

In [78]:
# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
#       print (list)
    except Exception as e:
      print(e)
    return list

In [79]:
df['text_clean'] = remove_trash(df['text'])

In [80]:
df[['text','text_clean']].head()

,text,text_clean
5,Как стало известно агентству Ассошиэйтед Пресс...,Как стало известно агентству Ассошиэйтед Пресс...
6,В зале игровых автоматов в третьем ярусе подзе...,В зале игровых автоматов в третьем ярусе подзе...
7,Япония приняла решение разморозить кредиты Рос...,Япония приняла решение разморозить кредиты Рос...
8,Британцы отмечают сегодня скорбную дату - втор...,Британцы отмечают сегодня скорбную дату вторую...
9,В понедельник директор департамента внешних св...,В понедельник директор департамента внешних св...


In [81]:
df.isna().sum()

url           0
title         0
text          0
topic         0
tags          0
date          0
text_clean    0
dtype: int64

In [ ]:
# df = df[~df.text_clean.isna()].copy()

In [ ]:
#токенизатор удалил мусор

In [83]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text_clean'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

In [84]:
clf3 = tree.DecisionTreeClassifier(max_depth=3)
clf3.fit(X_train,y_train)
pred3 = clf3.predict(X_test)
print('Качество модели по метрике F1 (токенизатор настроен и убран мусор)', f1_score(y_test,pred3,average='weighted'))

Качество модели по метрике F1 (токенизатор настроен и убран мусор) 0.38085878368230586


### попробуем лемматизацию

In [87]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=611f53427fb2c2ec8d2531990439eb7e87f876a57552b37a1b010bf52a69d363
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [88]:
import pymorphy2

In [89]:
morph = pymorphy2.MorphAnalyzer()

In [90]:
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [91]:
df.text

5        Как стало известно агентству Ассошиэйтед Пресс...
6        В зале игровых автоматов в третьем ярусе подзе...
7        Япония приняла решение разморозить кредиты Рос...
8        Британцы отмечают сегодня скорбную дату - втор...
9        В понедельник директор департамента внешних св...
                               ...                        
39994    Два тура президентских выборов во Франции прой...
39995    В Москве прошла конференция "РАО "ЕЭС России" ...
39996    Во вторник Верховный суд Якутии возобновил про...
39997    Во вторник Верховный суд Якутии отменил регист...
39998    Глава ГТК Михаил Ванин считает возможным увели...
Name: text, Length: 35223, dtype: object

In [92]:
t = []
text = 'Минфин предлагает увеличить размер штрафа'
for word in text.split():
    if len(word)<=3:
        continue
    p = morph.parse(word)[0]
    t.append(p.normal_form)

In [93]:
" ".join(t)

'минфин предлагать увеличить размер штраф'

In [94]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

CPU times: user 13min 43s, sys: 1.76 s, total: 13min 45s
Wall time: 13min 55s


In [ ]:
# пример чтения объекта из файла в оперативную память
with open('df_norma.pkl','rb') as f:
    df = pickle.load(f)

In [95]:
df.head()

,url,title,text,topic,tags,date,text_clean,text_clean_normal
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,Как стало известно агентству Ассошиэйтед Пресс...,как стать известно агентство ассошиэйтед пресс...
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31,В зале игровых автоматов в третьем ярусе подзе...,зал игровой автомат третий ярус подземный комп...
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31,Япония приняла решение разморозить кредиты Рос...,япония принять решение разморозить кредит росс...
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31,Британцы отмечают сегодня скорбную дату вторую...,британец отмечать сегодня скорбный дата вторую...
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31,В понедельник директор департамента внешних св...,понедельник директор департамент внешний связь...


In [96]:
df[['text','text_clean_normal']].tail(10)

,text,text_clean_normal
39989,Генеральный прокурор США Джон Эшкрофт (John As...,генеральный прокурор сша джон эшкрофт в вторни...
39990,Бывший артист известнейшей мальчиковый группы ...,бывший артист известный мальчиковый группа бра...
39991,Россия прогнозировала возможный односторонний ...,россия прогнозировать возможный односторонний ...
39992,"В Москве задержан особо опасный рецидивист, на...",москва задержать особо опасный рецидивист нахо...
39993,"""Делать сенсацию из того, что наша доблестная ...",делать сенсация из тот что наш доблестный мили...
39994,Два тура президентских выборов во Франции прой...,два тур президентский выборы в франция пройти ...
39995,"В Москве прошла конференция ""РАО ""ЕЭС России"" ...",москва пройти конференция рао еэс россия откры...
39996,Во вторник Верховный суд Якутии возобновил про...,в вторник верховный суд якутия возобновить про...
39997,Во вторник Верховный суд Якутии отменил регист...,в вторник верховный суд якутия отменить регист...
39998,Глава ГТК Михаил Ванин считает возможным увели...,глава гтк михаил ванин считать возможный увели...


### качество модели после лемматизации

In [98]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000)
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [ ]:
# text_transformer.transform(['Минфин предлагает увеличить размер'])

In [99]:
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)
pred_norm = clf_norm.predict(X_test)

In [ ]:
# clf_norm.predict(text_transformer.transform(['Минфин предлагает увеличить размер']))

In [100]:

print('Качество модели по метрике F1 после лемматизации', f1_score(y_test,pred_norm,average='weighted'))

Качество модели по метрике F1 после лемматизации 0.7465218956564681
